In [2]:
import pandas as pd
import numpy as np

from metrics import *
from ThreeDimensionalSTSSCAN import *

In [85]:
data = pd.read_csv("../data/Raw/Erk.csv", sep=";")

In [94]:
neg_data = data[data["Pvalue"]>0.5]
pos_data = data[(data["Pvalue"]<=0.01)]

In [99]:
pos_data[['CauseGene', 'EffectGene', 'Pvalue', '0min_neg_cause', '15min_cause', '30min_cause', '90min_cause',
       '120min_cause', '180min_cause', '210min_cause', '240min_cause',
       '360min_cause', '0min_neg_effect', '15min_effect', '30min_effect',
       '90min_effect', '120min_effect', '180min_effect', '210min_effect',
       '240min_effect', '360min_effect']].head()

,CauseGene,EffectGene,Pvalue,0min_neg_cause,15min_cause,30min_cause,90min_cause,120min_cause,180min_cause,210min_cause,...,360min_cause,0min_neg_effect,15min_effect,30min_effect,90min_effect,120min_effect,180min_effect,210min_effect,240min_effect,360min_effect
2256,MAPK1,AGTRAP,0.008,1.570461,-0.259946,0.461187,-1.821926,0.217530,-0.064708,0.709208,...,0.560650,1.343140,0.967046,0.550347,-0.396333,0.762589,-0.857914,-0.271856,-1.741207,0.751194
2257,MAPK1,AGTRAP,0.008,1.570461,0.540413,-0.058239,-0.384124,0.131689,0.143581,-0.430090,...,-0.447166,1.343140,0.141576,0.677390,-1.477123,-1.339167,-1.487985,0.477539,-0.191964,-0.440323
2258,MAPK1,AGTRAP,0.008,0.814767,-1.303725,0.780416,1.014104,0.493707,0.864028,-2.828600,...,0.487763,1.179496,1.347345,0.476586,-0.944036,-0.031326,-1.069956,-0.939858,-0.994739,-1.215449
2259,MAPK1,AGTRAP,0.008,0.814767,-1.453817,0.690254,0.163129,0.340793,-1.816556,0.369570,...,-0.016714,1.179496,0.093155,0.319289,-0.816700,-1.353020,0.005323,-0.141223,-0.072680,1.467384
2260,MAPK1,AGTRAP,0.008,-1.276183,1.060551,0.513360,-0.863885,-0.520006,0.386385,1.168475,...,0.610499,0.755433,1.313819,1.005074,-1.053026,-0.192625,-0.763318,-0.270895,-0.908896,0.380477


In [100]:
from sklearn.model_selection import train_test_split 

In [106]:
train, test = train_test_split(pos_data, train_size=0.9)

/home/a20125359/anaconda3/envs/Biclustering/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [110]:
print("train ", len(train), " test ", len(test))

train  237  test  27


In [131]:
neg_test = neg_data.sample(len(train) + len(test))

In [146]:
train.columns[14:]

Index(['0min_neg_effect', '15min_effect', '30min_effect', '90min_effect',
       '120min_effect', '180min_effect', '210min_effect', '240min_effect',
       '360min_effect'],
      dtype='object')

In [156]:
train_cause = train[train.columns[5:14]]
train_effect = train[train.columns[14:]]

In [163]:
train_trajectories = pd.DataFrame()

In [166]:
train_trajectories['0min'] = list(zip(train_cause['0min_neg_cause'], train_effect['0min_neg_effect']))
train_trajectories['15min'] = list(zip(train_cause['15min_cause'], train_effect['15min_effect']))
train_trajectories['30min'] = list(zip(train_cause['30min_cause'], train_effect['30min_effect']))
train_trajectories['90min'] = list(zip(train_cause['90min_cause'], train_effect['90min_effect']))
train_trajectories['120min'] = list(zip(train_cause['120min_cause'], train_effect['120min_effect']))
train_trajectories['180min'] = list(zip(train_cause['180min_cause'], train_effect['180min_effect']))
train_trajectories['210min'] = list(zip(train_cause['210min_cause'], train_effect['210min_effect']))
train_trajectories['240min'] = list(zip(train_cause['240min_cause'], train_effect['240min_effect']))
train_trajectories['360min'] = list(zip(train_cause['360min_cause'], train_effect['360min_effect']))

In [167]:
train_trajectories

,0min,15min,30min,90min,120min,180min,210min,240min,360min
0,"(1.570461219, 0.344553796)","(0.540413267, 0.676208773)","(-0.058239409, 0.123638251)","(-0.384124221, -1.049929117)","(0.131689062, -1.353716117)","(0.143581448, -1.083097724)","(-0.430089865, -0.724126064)","(-0.761856253, -1.685627107)","(-0.447165686, -1.747304454)"
1,"(1.570461219, 1.356048974)","(0.540413267, 0.955013538)","(-0.058239409, 1.646334273)","(-0.384124221, -0.10460869)","(0.131689062, -0.427351457)","(0.143581448, -0.073006472)","(-0.430089865, 0.484258421)","(-0.761856253, -0.208732025)","(-0.447165686, -0.412598416)"
2,"(0.814766536, 0.266287459)","(-1.303725398, -0.68488785)","(0.780415747, -0.221691578)","(1.014103853, 0.750970115)","(0.493707044, 0.16117456)","(0.864028135, 0.796638264)","(-2.828600353, 0.974805322)","(1.306187104, 1.614564696)","(0.487763142, 2.018363877)"
3,"(-0.131957365, 0.980872463)","(-0.069708394, 0.506057258)","(0.436723813, 0.997692871)","(0.633419321, 0.37128718)","(0.298561387, -1.172895046)","(0.410703006, -0.439438817)","(-0.995972794, -0.699049671)","(-0.198591966, 0.22138302)","(-0.086425808, -2.752785602)"
4,"(-0.297718156, 1.21508935)","(0.340381074, 3.78389795)","(0.764473299, 1.538866487)","(1.690905687, 0.902490886)","(0.500070056, -0.544110622)","(-0.141858448, -0.437356623)","(0.7180593, -1.265228466)","(-4.13741482, -1.348518849)","(-0.026669304, -1.758837311)"
5,"(0.246209847, 0.724090317)","(0.871711629, -0.128158205)","(1.345559794, 0.421639611)","(2.682876158, -1.304117922)","(0.665652809, -0.756851815)","(0.200693652, -1.048316872)","(0.256373885, -0.85549152)","(-0.116288574, -1.090157516)","(0.226120933, -1.821169991)"
6,"(-0.297718156, -0.929106073)","(-0.098115403, 0.870416927)","(0.488875352, -0.369249653)","(-0.532410322, 0.280191282)","(-2.063256734, -0.212138567)","(0.144287109, -1.378583689)","(0.482835554, -1.156738331)","(0.754663737, -1.000777462)","(1.734924441, 0.15591201)"
7,"(-1.276182559, -0.929106073)","(-3.544150253, 0.870416927)","(0.38607826, -0.369249653)","(0.293890246, 0.280191282)","(0.756262003, -0.212138567)","(0.176533551, -1.378583689)","(0.14655329, -1.156738331)","(-0.687439361, -1.000777462)","(1.832161077, 0.15591201)"
8,"(1.570461219, -0.25880353)","(-0.259945662, -0.10396347)","(0.461186883, 0.039940589)","(-1.821925634, -0.21284122)","(0.217530138, 0.871548858)","(-0.064708141, 1.667837762)","(0.70920845, 2.095155457)","(0.849053723, 1.44675436)","(0.560649686, 1.831465349)"
9,"(0.246209847, -0.058146798)","(0.871711629, 0.264768701)","(1.345559794, 0.272730661)","(2.682876158, 1.630527534)","(0.665652809, 0.605786526)","(0.200693652, 0.049190652)","(0.256373885, 0.010502444)","(-0.116288574, -0.928628969)","(0.226120933, -1.462997374)"
